In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import postgres

### Import Artists CSV into DataFrame & Clean

In [2]:
#Import artist CSV file
artist_csv_file = "Resources/artists.csv"
artists_df = pd.read_csv(artist_csv_file, encoding='utf8')

In [83]:
#Isolate only artists with >5k followers
popular_artists_df = artists_df.loc[(artists_df["followers"] > 5000)]

In [84]:
#Rename columns to align to pgAdmin table
popular_artists_df = popular_artists_df.rename(columns = {"name":"artist", "id": "artist_id"})

### Tracks CSV into DataFrame & Clean

In [10]:
tracks_csv_file = "Resources/data_tracks.csv"
tracks_raw_df = pd.read_csv(tracks_csv_file, encoding='utf8')

In [18]:
#Clear out data pre-1980
tracks_post1980 = tracks_raw_df.loc[(tracks_raw_df["year"] >= 1980)]

In [19]:
#Clear out unnecessary columns, reset index after discarding data in previous step
tracks_df = tracks_post1980[['year', 'artists', 'danceability','duration_ms','energy',
                           'explicit','id','loudness','name','popularity','tempo']].copy()

In [20]:
#Rename columns to align to pgAdmin table
tracks_df = tracks_df.rename(columns = {"artists":"artist"})
tracks_df.head()

,year,artist,danceability,duration_ms,energy,explicit,id,loudness,name,popularity,tempo
11606,1980,AC/DC,0.310,255493,0.700,0,08mG3Y1vljYA6bvDt4Wqkj,-5.678,Back In Black,84,188.386
11607,1980,Paul McCartney,0.774,227653,0.511,0,1SV1fxF65n9NhRHp3KlBuu,-8.914,Wonderful Christmastime - Edited Version / Rem...,79,94.736
11608,1980,Queen,0.933,214653,0.528,0,57JVGBtBLCfHw2muk5416J,-6.472,Another One Bites The Dust - Remastered 2011,73,109.975
11609,1980,Daryl Hall & John Oates,0.751,190627,0.501,0,4o6BgsqLIBViaGVbx5rbRk,-12.151,You Make My Dreams (Come True),78,167.057
11610,1980,AC/DC,0.532,210173,0.767,0,2SiXAy7TuUkycRVbbWDEpo,-5.509,You Shook Me All Night Long,80,127.361


### Merge Dataframes

In [85]:
#Merge the two dataframes using artist name - 
    #duplicate artists mostly cleared out by the popular argument requiring >5k followers
tracks_and_follower_df = pd.merge(tracks_df, popular_artists_df[['artist','followers', 'artist_id']],on='artist', how='left')

In [94]:
#drop any rows containing NaN values
clean_tracks_and_follower_df = tracks_and_follower_df.dropna(how='any')

In [95]:
#drop any rows containing duplicate id values
cleaner_tracks = clean_tracks_and_follower_df.drop_duplicates(['id'],keep=False)

In [96]:
#Create new artist table
clean_artists = cleaner_tracks[['artist_id', 'artist', 'followers']].copy()

In [97]:
#tidy up any duplicate s in dataset
clean_artists.drop_duplicates(keep="first",inplace=True)
clean_artists.sort_values(by=["followers"], inplace=True,ascending=False)
clean_artists.drop_duplicates(['artist_id'],keep="first",inplace=True)

In [99]:
#remove artist followers column from the tracks dataset to avoid confusion with track specific popularity
cleanest_tracks_df = cleaner_tracks.drop(columns='followers',inplace=False)
cleanest_tracks_df.to_csv("trackcheck.csv")

### Connect to local database

In [65]:
rds_connection_string = f"postgres:{postgres}localhost:5432/music-masters_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [100]:
engine.table_names()

['spotify_artists', 'spotify_tracks']

### Use pandas to load csv converted DataFrame into database

In [101]:
clean_artists.to_sql(name='spotify_artists', con=engine, if_exists='append', index=False)

In [102]:
cleanest_tracks_df.to_sql(name='spotify_tracks', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the spotify table
* NOTE: can also check using pgAdmin

In [103]:
pd.read_sql_query('select * from spotify_tracks', con=engine).head()

,year,artist,danceability,duration_ms,energy,explicit,id,loudness,name,popularity,tempo,artist_id
0,1980,AC/DC,0,255493,1,0,08mG3Y1vljYA6bvDt4Wqkj,-6,Back In Black,84,188,711MCceyCBcFnzjGY4Q7Un
1,1980,Paul McCartney,1,227653,1,0,1SV1fxF65n9NhRHp3KlBuu,-9,Wonderful Christmastime - Edited Version / Rem...,79,95,4STHEaNw4mPZ2tzheohgXB
2,1980,Queen,1,214653,1,0,57JVGBtBLCfHw2muk5416J,-6,Another One Bites The Dust - Remastered 2011,73,110,1dfeR4HaWDbWqFHLkxsg1d
3,1980,Daryl Hall & John Oates,1,190627,1,0,4o6BgsqLIBViaGVbx5rbRk,-12,You Make My Dreams (Come True),78,167,77tT1kLj6mCWtFNqiOmP9H
4,1980,AC/DC,1,210173,1,0,2SiXAy7TuUkycRVbbWDEpo,-6,You Shook Me All Night Long,80,127,711MCceyCBcFnzjGY4Q7Un
